<a href="https://colab.research.google.com/github/MoqiSheng/MoqiSheng.github.io/blob/main/250714_InfoNCE_256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.0 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity


class InfoNCELoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(InfoNCELoss, self).__init__()
        self.temperature = temperature

    def forward(self, anchor, positive, negatives):
        batch_size = anchor.size(0)
        device = anchor.device

        # 计算相似度
        pos_sim = F.cosine_similarity(anchor, positive, dim=1) / self.temperature  # [batch_size]

        # 负样本相似度
        anchor_expanded = anchor.unsqueeze(1)  # [batch_size, 1, feature_dim]
        neg_sim = F.cosine_similarity(anchor_expanded, negatives, dim=2) / self.temperature  # [batch_size, num_negatives]

        # 拼接正负样本相似度
        logits = torch.cat([pos_sim.unsqueeze(1), neg_sim], dim=1)  # [batch_size, 1+num_negatives]

        # 标签：正样本索引为0
        labels = torch.zeros(batch_size, dtype=torch.long, device=device)

        # 计算交叉熵损失
        loss = F.cross_entropy(logits, labels)

        return loss


def compute_node_similarities(embeddings, batch_size=1000):
    num_nodes = embeddings.shape[0]
    device = embeddings.device

    # 转换为numpy进行相似度计算（CPU上更稳定）
    embeddings_cpu = embeddings.detach().cpu().numpy()

    # 分批计算相似度
    similarity_matrix = np.zeros((num_nodes, num_nodes))

    for i in range(0, num_nodes, batch_size):
        end_i = min(i + batch_size, num_nodes)
        batch_embeddings = embeddings_cpu[i:end_i]

        # 计算当前批次与所有节点的相似度
        batch_similarities = cosine_similarity(batch_embeddings, embeddings_cpu)
        similarity_matrix[i:end_i] = batch_similarities

    return similarity_matrix


def find_positive_negative_samples(similarity_matrix,
                                 neg_min_sim=0.1, neg_max_sim=0.5, max_negatives=768,
                                 pos_min_sim=0.9, max_positives=50, random_seed=42):
    """
    同时计算正样本和负样本索引

    Args:
        similarity_matrix: 相似度矩阵
        neg_min_sim, neg_max_sim: 负样本相似度范围
        max_negatives: 最大负样本数量
        pos_min_sim: 正样本最小相似度阈值
        max_positives: 最大正样本数量
        random_seed: 随机种子

    Returns:
        negative_indices: 负样本索引字典
        positive_indices: 正样本索引字典
    """
    np.random.seed(random_seed)

    num_nodes = similarity_matrix.shape[0]
    negative_indices = {}
    positive_indices = {}

    for i in range(num_nodes):
        sim_row = similarity_matrix[i]

        # 找负样本：相似度在[neg_min_sim, neg_max_sim]范围内
        neg_mask = (sim_row >= neg_min_sim) & (sim_row <= neg_max_sim)
        neg_candidates = np.where(neg_mask)[0]
        neg_candidates = neg_candidates[neg_candidates != i]  # 排除自己

        if len(neg_candidates) > max_negatives:
            neg_candidates = np.random.choice(neg_candidates, max_negatives, replace=False)

        negative_indices[i] = neg_candidates.tolist()

        # 找正样本：相似度大于pos_min_sim
        pos_mask = sim_row > pos_min_sim
        pos_candidates = np.where(pos_mask)[0]
        pos_candidates = pos_candidates[pos_candidates != i]  # 排除自己

        if len(pos_candidates) > max_positives:
            pos_candidates = np.random.choice(pos_candidates, max_positives, replace=False)

        positive_indices[i] = pos_candidates.tolist()

    return negative_indices, positive_indices


def get_positive_samples(embeddings, edge_index, node_indices, positive_indices=None, num_extra_positives=3, random_seed=42):
    """
    获取正样本，包括邻居节点和额外的高相似度正样本

    Args:
        embeddings: 节点嵌入
        edge_index: 边索引
        node_indices: 节点索引列表
        positive_indices: 额外正样本索引字典
        num_extra_positives: 额外正样本数量
        random_seed: 随机种子
    """
    device = embeddings.device
    positive_samples = []

    # 设置随机种子
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)

    # 构建邻接列表
    num_nodes = embeddings.size(0)
    adj_list = {i: [] for i in range(num_nodes)}

    for i in range(edge_index.size(1)):
        src, dst = edge_index[0, i].item(), edge_index[1, i].item()
        adj_list[src].append(dst)

    for node_id in node_indices:
        # 获取原始邻居节点
        neighbors = adj_list[node_id]
        neighbor_embeddings_list = []

        if len(neighbors) > 0:
            neighbor_embeddings_list.append(embeddings[neighbors])  # [num_neighbors, feature_dim]

        # 添加额外的高相似度正样本（如果有的话）
        if positive_indices is not None and node_id in positive_indices:
            extra_pos_candidates = positive_indices[node_id]

            # 从候选中排除已经连接的邻居节点
            extra_pos_candidates = [idx for idx in extra_pos_candidates if idx not in neighbors]

            if len(extra_pos_candidates) > 0:
                # 随机选择额外的正样本
                num_to_select = min(num_extra_positives, len(extra_pos_candidates))
                selected_extra_pos = np.random.choice(extra_pos_candidates, num_to_select, replace=False)
                neighbor_embeddings_list.append(embeddings[selected_extra_pos])

        # 计算正样本的平均嵌入
        if len(neighbor_embeddings_list) > 0:
            all_neighbor_embeddings = torch.cat(neighbor_embeddings_list, dim=0)
            positive_sample = torch.mean(all_neighbor_embeddings, dim=0)
        else:
            # 如果没有邻居和额外正样本，使用自身嵌入
            positive_sample = embeddings[node_id]

        positive_samples.append(positive_sample)

    return torch.stack(positive_samples)  # [len(node_indices), feature_dim]


def get_negative_samples(embeddings, negative_indices, node_indices, num_negatives=256, random_seed=42):
    """获取负样本"""
    np.random.seed(random_seed)
    negative_samples = []

    for node_id in node_indices:
        if node_id in negative_indices and len(negative_indices[node_id]) > 0:
            neg_indices = negative_indices[node_id]

            # 随机选择负样本
            if len(neg_indices) >= num_negatives:
                selected_negatives = np.random.choice(neg_indices, num_negatives, replace=False)
            else:
                selected_negatives = neg_indices + np.random.choice(neg_indices,
                                                                  num_negatives - len(neg_indices),
                                                                  replace=True).tolist()

            neg_embeddings = embeddings[selected_negatives]  # [num_negatives, feature_dim]
        else:
            # 如果没有负样本，随机选择
            num_nodes = embeddings.size(0)
            random_indices = np.random.choice(num_nodes, num_negatives, replace=False)
            neg_embeddings = embeddings[random_indices]

        negative_samples.append(neg_embeddings)

    return torch.stack(negative_samples)  # [len(node_indices), num_negatives, feature_dim]


def compute_infonce_loss_batch(embeddings, edge_index, negative_indices, positive_indices,
                              node_batch, infonce_criterion, num_negatives=256, num_extra_positives=3, epoch=0):
    """
    计算InfoNCE损失（批处理版本）

    Args:
        embeddings: 节点嵌入
        edge_index: 边索引
        negative_indices: 负样本索引
        positive_indices: 正样本索引
        node_batch: 节点批次
        infonce_criterion: InfoNCE损失函数
        num_negatives: 负样本数量
        num_extra_positives: 额外正样本数量
        epoch: 当前epoch（用作随机种子的一部分）
    """
    # 使用epoch作为随机种子的一部分，确保可复现性
    random_seed = 42 + epoch

    # 获取锚点嵌入
    anchor_embeddings = embeddings[node_batch]  # [batch_size, feature_dim]

    # 获取正样本（包括额外的高相似度正样本）
    positive_embeddings = get_positive_samples(embeddings, edge_index, node_batch,
                                             positive_indices, num_extra_positives, random_seed)

    # 获取负样本
    negative_embeddings = get_negative_samples(embeddings, negative_indices,
                                             node_batch, num_negatives, random_seed)

    # 计算InfoNCE损失
    batch_loss = infonce_criterion(anchor_embeddings, positive_embeddings, negative_embeddings)

    return batch_loss

In [4]:
import pickle
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import collections
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from sklearn import linear_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error, \
    top_k_accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from transformers import AutoModel
from torch_geometric.nn import GATConv, GCNConv
from collections import Counter
from torch_geometric.data import Data


class SVFeatureBlock(nn.Module):
    def __init__(self, input_size=512, hidden_size=512, mode='mean'):
        super(SVFeatureBlock, self).__init__()
        self.mode = mode
        self.input_size = input_size
        self.hidden_size = hidden_size

        if mode == 'lstm':
            self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
            nn.init.orthogonal_(self.lstm.weight_ih_l0)
            nn.init.orthogonal_(self.lstm.weight_hh_l0)
        elif mode == 'bi-lstm':
            self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True,
                                bidirectional=True)
        elif self.mode == "gru":
            self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        elif mode == 'rnn':
            self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)

    def forward(self, sv):
        sv_list = []
        for x_tmp in sv:
            if self.mode == "mean":
                if x_tmp.dim() != 1:
                    out_put = torch.mean(x_tmp, dim=0)
            elif self.mode == "sum":
                if x_tmp.dim() != 1:
                    out_put = torch.sum(x_tmp, dim=0)
            elif self.mode == "max":
                if x_tmp.dim() != 1:
                    out_put = torch.max(x_tmp, dim=0).values
            elif self.mode == "lstm":
                out_put, (h_n, c_n) = self.lstm(x_tmp.view(1, -1, self.input_size))
                out_put = out_put[:, -1, :]
                out_put = torch.squeeze(out_put)
            else:
                pass

            sv_list.append(out_put)
        x = torch.stack(sv_list)  # 拼接,(batch,512)
        return x


def weights_init_1(m):
    seed = 20
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.nn.init.xavier_uniform_(m.weight, gain=1)


def weights_init_2(m):
    seed = 20
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.nn.init.xavier_uniform_(m.weight, gain=1)
    torch.nn.init.constant_(m.bias, 0)


class Attention_Soft(nn.Module):
    def __init__(self, in_size, hidden_size=32):
        super(Attention_Soft, self).__init__()

        self.l1 = torch.nn.Linear(in_size, hidden_size, bias=True)
        self.ac = nn.Sigmoid()
        self.l2 = torch.nn.Linear(in_size, hidden_size, bias=False)
        self.l3 = torch.nn.Linear(int(hidden_size), 1, bias=False)

        weights_init_2(self.l1)
        weights_init_1(self.l2)
        weights_init_1(self.l3)

    def forward(self, z):
        w1 = self.l1(torch.mean(z, dim=1).unsqueeze(1))
        w2 = self.l2(z)
        w = self.ac(w1 + w2)
        w = self.l3(w)
        beta = torch.softmax(w, dim=1)

        return (beta * z).sum(1)


class Text_MLP(nn.Module):
    def __init__(self, input_dim=4096, hidden_dim=2048, output_dim=768):
        super(Text_MLP, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_dim, output_dim)
        weights_init_2(self.layer1)
        weights_init_2(self.layer2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x


class SV_GAT(nn.Module):
    def __init__(self, args):
        super(SV_GAT, self).__init__()
        self.args = args
        self.length = list(np.load('/content/drive/MyDrive/USPM_edege_add/data/length.npy'))
        pretrain_sv_path = args.pretrain_sv_path
        pretrain_scn_path = args.pretrain_scn_path
        self.sv_embedding = torch.load(pretrain_sv_path, map_location=torch.device(args.device))
        self.scn_embedding = torch.load(pretrain_scn_path, map_location=torch.device(args.device))

        self.text_mlp = Text_MLP(input_dim=4096, hidden_dim=2048, output_dim=768)

        self.sv_agg = SVFeatureBlock(input_size=768, hidden_size=768, mode=args.mode)

        self.attention_soft = Attention_Soft(in_size=768)

        self.gat = GAT(input_dim=768, hidden_dim=64, output_dim=10, heads=8, args=args, drop=0.6)

        self.gat_poi = GAT_P(input_dim=768, hidden_dim=64, output_dim=4, heads=8, args=args)

    def forward(self, epoch=0, test_results=None):
        sv_features = self.sv_embedding
        street_list = list(torch.split(sv_features, self.length, dim=0))
        sv_aggre = self.sv_agg(street_list)
        sv_embedding = sv_aggre
        scn_embedding = self.text_mlp(self.scn_embedding)  # Reduce text embedding to 768 dim
        street_embedding = self.attention_soft(torch.stack([scn_embedding, sv_embedding], dim=1))

        if self.args.downstream == 'poi':
            gat_loss, infonce_loss, out = self.gat_poi(street_embedding, epoch, test_results)
        else:
            gat_loss, infonce_loss, s_emb1, out = self.gat(street_embedding, epoch, test_results)

        return gat_loss, infonce_loss, out, street_embedding

    def test(self, out):
        if self.args.downstream == 'poi':
            acc, f1_score_test, mrr_test, num, pred_out = self.gat_poi.test(out)
            return acc, f1_score_test, mrr_test, 1, 1, 1, num, pred_out
        else:
            a1, a3, a5, a10, f1, mrr, num, pred_out = self.gat.test(out)
            return a1, a3, a5, a10, f1, mrr, num, pred_out


class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads, args, drop=0.6):
        super().__init__()
        torch.manual_seed(0)
        self.args = args
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, concat=False, heads=10, dropout=0.6)

        self.elu = nn.ELU()
        self.drop1 = nn.Dropout(p=drop)
        self.drop2 = nn.Dropout(p=0.6)

        # InfoNCE相关
        self.infonce_criterion = InfoNCELoss(temperature=0.5)
        self.negative_indices = None
        self.positive_indices = None  # 新增正样本索引
        self.infonce_triggered = False
        self.similarity_computed = False
        self.result_dir = 'infonce_data'

        # 触发条件：num=10且f1>=0.44
        self.trigger_num_threshold = 10
        self.trigger_f1_threshold = 0.44

        self.edge_index = torch.load('/content/drive/MyDrive/USPM_edege_add/data/edge_index.pt').t().contiguous().to(args.device)
        self.y = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_edege_add/data/function/label_all_function.npy', allow_pickle=True)).long().to(args.device)
        self.train_mask = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_edege_add/data/function/label_mask.npy', allow_pickle=True)).to(args.device)
        self.mask = torch.load('/content/drive/MyDrive/USPM_edege_add/data/function/test_mask.pt')
        self.y_testlabel = np.load('/content/drive/MyDrive/USPM_edege_add/data/function/label_all_function.npy')[self.mask]

    def forward(self, street_embedding, epoch=0, test_results=None):
        street_embedding_0 = self.drop1(street_embedding)
        street_embedding_1 = self.conv1(street_embedding_0, self.edge_index)
        street_embedding_2 = self.elu(street_embedding_1)
        street_embedding_2 = self.drop2(street_embedding_2)
        street_embedding_2 = self.conv2(street_embedding_2, self.edge_index)

        cross_criterion = torch.nn.CrossEntropyLoss()
        loss_su = cross_criterion(street_embedding_2[self.train_mask], self.y[self.train_mask])

        # InfoNCE损失计算
        infonce_loss = torch.tensor(0.0, device=street_embedding.device)

        # 检查是否满足触发条件
        if test_results and not self.similarity_computed:
            num = test_results.get('num', 0)
            f1 = test_results.get('f1', 0.0)

            if num >= self.trigger_num_threshold and f1 >= self.trigger_f1_threshold:
                # 满足条件，计算并保存正负样本索引
                print(f"Triggering InfoNCE for GAT at epoch {epoch}: num={num}, f1={f1:.4f}")
                print("Computing positive and negative samples...")
                similarity_matrix = compute_node_similarities(street_embedding, batch_size=500)

                # 同时计算正负样本索引
                self.negative_indices, self.positive_indices = find_positive_negative_samples(
                    similarity_matrix,
                    neg_min_sim=0.1, neg_max_sim=0.5, max_negatives=768,
                    pos_min_sim=0.9, max_positives=50,
                    random_seed=42 + epoch
                )

                self.similarity_computed = True
                self.infonce_triggered = True

        if self.infonce_triggered and self.negative_indices is not None and self.positive_indices is not None:
            # 已经触发InfoNCE，计算损失
            num_nodes = street_embedding_1.size(0)
            batch_size = 256  # 分批处理
            total_infonce_loss = 0.0
            num_batches = 0

            for i in range(0, num_nodes, batch_size):
                end_idx = min(i + batch_size, num_nodes)
                node_batch = list(range(i, end_idx))

                try:
                    batch_loss = compute_infonce_loss_batch(
                        street_embedding_1, self.edge_index,
                        self.negative_indices, self.positive_indices,
                        node_batch, self.infonce_criterion,
                        num_negatives=256, num_extra_positives=3, epoch=epoch
                    )
                    total_infonce_loss += batch_loss
                    num_batches += 1
                except Exception as e:
                    print(f"Error in InfoNCE computation for batch {i}-{end_idx}: {e}")
                    continue

            if num_batches > 0:
                infonce_loss = total_infonce_loss / num_batches

        return loss_su, infonce_loss, street_embedding_1, street_embedding_2

    def test(self, out):
        pred = out.argmax(dim=1)
        pred = pd.DataFrame({'Type': torch.Tensor.cpu(pred).numpy()})

        predictions_test_dim = torch.Tensor.cpu(out[self.mask]).argmax(dim=1).detach().numpy()
        predictions_test = torch.Tensor.cpu(out[self.mask]).detach().numpy()

        A1 = top_k_accuracy_score(self.y_testlabel, predictions_test, k=1, labels=range(10))
        A3 = top_k_accuracy_score(self.y_testlabel, predictions_test, k=3, labels=range(10))
        A5 = top_k_accuracy_score(self.y_testlabel, predictions_test, k=5, labels=range(10))
        print(f'A1={A1}\t A3={A3}\t A5={A5} ')

        precision_score_test = precision_score(self.y_testlabel, predictions_test_dim, average="weighted")
        f1_score_test = f1_score(self.y_testlabel, predictions_test_dim, average="weighted")
        mrr_test = compute_mrr(self.y_testlabel, predictions_test)
        result = Counter(pred['Type'].values.tolist())
        num = len(result)
        print(
            f'precision={precision_score_test}, f1={f1_score_test}, mrr={mrr_test},num={num}')

        print(result)
        return A1, A3, A5, 1, f1_score_test, mrr_test, num, out


class GAT_P(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads, args, drop=0.6):
        super().__init__()
        self.args = args
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=drop)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, concat=False, heads=4, dropout=drop)
        self.elu = nn.ELU()
        self.drop1 = nn.Dropout(p=drop)
        self.drop2 = nn.Dropout(p=drop)

        # InfoNCE相关
        self.infonce_criterion = InfoNCELoss(temperature=0.5)
        self.negative_indices = None
        self.positive_indices = None  # 新增正样本索引
        self.infonce_triggered = False
        self.similarity_computed = False
        self.result_dir = 'infonce_data'

        # 触发条件：num=4且f1>=0.38
        self.trigger_num_threshold = 4
        self.trigger_f1_threshold = 0.38

        self.edge_index = torch.load('/content/drive/MyDrive/USPM_edege_add/data/edge_index.pt').t().contiguous().to(args.device)

        self.y = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_edege_add/data/poi/label_all_poi_level.npy', allow_pickle=True)).long().to(args.device)
        self.train_mask = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_edege_add/data/poi/label_mask_poi_level.npy', allow_pickle=True)).to(args.device)
        self.test_mask = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_edege_add/data/poi/test_mask_poi_level.npy', allow_pickle=True)).to(args.device)

        self.mask = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_edege_add/data/poi/test_mask_poi_level.npy', allow_pickle=True))

        self.y_testlabel = np.load('/content/drive/MyDrive/USPM_edege_add/data/poi/label_all_poi_level.npy')[self.mask]

    def forward(self, street_embedding, epoch=0, test_results=None):
        street_embedding_0 = self.drop1(street_embedding)
        street_embedding_1 = self.conv1(street_embedding_0, self.edge_index)
        street_embedding_2 = self.elu(street_embedding_1)
        street_embedding_2 = self.drop2(street_embedding_2)
        street_embedding_2 = self.conv2(street_embedding_2, self.edge_index)

        cross_criterion = torch.nn.CrossEntropyLoss()
        loss_su = cross_criterion(street_embedding_2[self.train_mask], self.y[self.train_mask])

        # InfoNCE损失计算
        infonce_loss = torch.tensor(0.0, device=street_embedding.device)

        # 检查是否满足触发条件
        if test_results and not self.similarity_computed:
            num = test_results.get('num', 0)
            f1 = test_results.get('f1', 0.0)

            if num >= self.trigger_num_threshold and f1 >= self.trigger_f1_threshold:
                # 满足条件，计算并保存正负样本索引
                print(f"Triggering InfoNCE for GAT_P at epoch {epoch}: num={num}, f1={f1:.4f}")
                print("Computing positive and negative samples...")
                similarity_matrix = compute_node_similarities(street_embedding, batch_size=500)

                # 同时计算正负样本索引
                self.negative_indices, self.positive_indices = find_positive_negative_samples(
                    similarity_matrix,
                    neg_min_sim=0.1, neg_max_sim=0.5, max_negatives=768,
                    pos_min_sim=0.9, max_positives=50,
                    random_seed=42 + epoch
                )


                self.similarity_computed = True
                self.infonce_triggered = True

        if self.infonce_triggered and self.negative_indices is not None and self.positive_indices is not None:
            # 已经触发InfoNCE，计算损失
            num_nodes = street_embedding_1.size(0)
            batch_size = 256  # 分批处理
            total_infonce_loss = 0.0
            num_batches = 0

            for i in range(0, num_nodes, batch_size):
                end_idx = min(i + batch_size, num_nodes)
                node_batch = list(range(i, end_idx))

                try:
                    batch_loss = compute_infonce_loss_batch(
                        street_embedding_1, self.edge_index,
                        self.negative_indices, self.positive_indices,
                        node_batch, self.infonce_criterion,
                        num_negatives=256, num_extra_positives=3, epoch=epoch
                    )
                    total_infonce_loss += batch_loss
                    num_batches += 1
                except Exception as e:
                    print(f"Error in InfoNCE computation for batch {i}-{end_idx}: {e}")
                    continue

            if num_batches > 0:
                infonce_loss = total_infonce_loss / num_batches

        return loss_su, infonce_loss, street_embedding_2

    def test(self, out):
        pred = out.argmax(dim=1)
        correct = pred[self.test_mask] == self.y[self.test_mask]
        acc = int(correct.sum()) / int(self.test_mask.sum())

        pred = pd.DataFrame({'Type': torch.Tensor.cpu(pred).numpy()})

        predictions_test_dim = torch.Tensor.cpu(out[self.mask]).argmax(dim=1).detach().numpy()
        predictions_test = torch.Tensor.cpu(out[self.mask]).detach().numpy()
        f1_score_test = f1_score(self.y_testlabel, predictions_test_dim, average="macro")
        mrr_test = compute_mrr(self.y_testlabel, predictions_test)
        result = Counter(pred['Type'].values.tolist())
        num = len(result)
        print(
            f'acc={acc}, f1={f1_score_test}, mrr={mrr_test},num={num}')

        print(result)
        return acc, f1_score_test, mrr_test, num, out


def compute_mrr(true_labels, machine_preds):
    """Compute the MRR """
    rr_total = 0.0
    for i in range(len(true_labels)):
        if true_labels[i] == 403:
            continue
        ranklist = list(np.argsort(machine_preds[i])[::-1])
        rank = ranklist.index(true_labels[i]) + 1
        rr_total = rr_total + 1.0 / rank
    mrr = rr_total / len(true_labels)
    return mrr

In [ ]:
import itertools
import os
import random
import math
import torch
import numpy as np
import argparse
import warnings
from datetime import datetime
# from model import SV_GAT

warnings.filterwarnings('ignore')
parser = argparse.ArgumentParser()
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

parser.add_argument('--device', type=str, default='cuda:0', help='gpu device ids')
parser.add_argument('--print_num', type=int, default=1, help='gap of print evaluations')
parser.add_argument("--print_epoch", type=int, default=0, help="Start print epoch")
parser.add_argument("--start_epoch", type=int, default=0, help="Start epoch")
parser.add_argument("--current_epoch", type=int, default=0, help="Current epoch")
parser.add_argument("--epochs", type=int, default=200, help="Epochs")
parser.add_argument("--seed", type=int, default=42, help="random seed.")
parser.add_argument("--rounds", type=int, default=5, help="number of training rounds")
parser.add_argument("--mode", type=str, default='lstm', help="aggression function.")

args = parser.parse_args([])

def trainer(args, model, optimizer1, optimizer2, optimizer3, optimizer4, epoch, test_results=None):
    loss_epoch = []
    loss_su_epoch = []
    infonce_loss_epoch = []
    total_loss_epoch = []

    model.train()
    optimizer1.zero_grad() # attention
    optimizer2.zero_grad() # sv_agg(lstm)
    optimizer3.zero_grad() # gat
    optimizer4.zero_grad() # text_mlp

    # 传入epoch和test_results参数以便模型检查InfoNCE触发条件
    gnn_loss, infonce_loss, pre_out, street_embedding = model(epoch, test_results)

    # 记录各种损失
    loss_su_epoch.append(gnn_loss.item())
    infonce_loss_epoch.append(infonce_loss.item())

    # 计算总损失
    if args.downstream == 'poi':
        infonce_triggered = model.gat_poi.infonce_triggered
    else:
        infonce_triggered = model.gat.infonce_triggered

    if not infonce_triggered:
        # InfoNCE未触发前，只使用原始损失
        total_loss = gnn_loss
    else:
        # InfoNCE触发后，使用加权损失
        total_loss = 0.8 * gnn_loss + 0.2 * infonce_loss

    total_loss_epoch.append(total_loss.item())
    loss_epoch.append(total_loss.item())  # 保持兼容性

    total_loss.backward()

    optimizer1.step()
    optimizer2.step()
    optimizer3.step()
    optimizer4.step()

    if epoch % args.print_num == 0:
        if not infonce_triggered:
            print(f"TrainEpoch [{epoch + 1}/{args.epochs}]\t loss_su:{np.mean(loss_su_epoch):.6f}")
        else:
            print(f"TrainEpoch [{epoch + 1}/{args.epochs}]\t total_loss:{np.mean(total_loss_epoch):.6f}\t "
                  f"loss_su:{np.mean(loss_su_epoch):.6f}\t infonce_loss:{np.mean(infonce_loss_epoch):.6f}")

    return np.mean(loss_epoch), pre_out, street_embedding

def test(args, model, epoch, round_num, result_dir):
    with torch.no_grad():
        model.eval()
        _, _, out, _ = model(epoch)  # 测试时不传入test_results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        os.makedirs(result_dir, exist_ok=True)
        if args.downstream == 'poi':
            acc, f1, mrr, _, _, _, num, pred_out = model.test(out)
            result = {
                'epoch': epoch + 1,
                'acc': acc,
                'f1': f1,
                'mrr': mrr,
                'num': num
            }
            return acc, f1, mrr, 1, 1, 1, num, pred_out, result
        else:
            a1, a3, a5, a10, f1, mrr, num, pred_out = model.test(out)
            result = {
                'epoch': epoch + 1,
                'a1': a1,
                'a3': a3,
                'a5': a5,
                'a10': a10,
                'f1': f1,
                'mrr': mrr,
                'num': num
            }
            return a1, a3, a5, a10, f1, mrr, num, pred_out, result

def calculate_best_results(result_dir, downstream):
    # Collect round files
    round_files = [f for f in os.listdir(result_dir) if f.startswith('round_') and f.endswith('.npy')]
    if not round_files:
        return None

    # Load all results and group by epoch
    epoch_results = {}
    for round_file in round_files:
        round_data = np.load(os.path.join(result_dir, round_file), allow_pickle=True).item()
        for epoch_data in round_data['epochs']:
            epoch = epoch_data['epoch']
            if epoch not in epoch_results:
                epoch_results[epoch] = []
            epoch_results[epoch].append(epoch_data)

    # Compute per-epoch averages across rounds
    epoch_averages = {}
    best_f1 = -1
    best_epoch = None

    if downstream == 'poi':
        all_results = []
        for epoch in epoch_results:
            accs = [r['acc'] for r in epoch_results[epoch]]
            f1s = [r['f1'] for r in epoch_results[epoch]]
            mrrs = [r['mrr'] for r in epoch_results[epoch]]
            epoch_averages[epoch] = {
                'avg_acc': np.mean(accs),
                'avg_f1': np.mean(f1s),
                'avg_mrr': np.mean(mrrs)
            }
            all_results.extend(epoch_results[epoch])
            if epoch_averages[epoch]['avg_f1'] > best_f1:
                best_f1 = epoch_averages[epoch]['avg_f1']
                best_epoch = epoch

        # Store best epoch results and overall max metrics
        best_results = {
            'best_epoch': best_epoch,
            'best_epoch_avg_acc': epoch_averages[best_epoch]['avg_acc'],
            'best_epoch_avg_f1': epoch_averages[best_epoch]['avg_f1'],
            'best_epoch_avg_mrr': epoch_averages[best_epoch]['avg_mrr'],
            'overall_best_acc': max([r['acc'] for r in all_results]),
            'overall_best_f1': max([r['f1'] for r in all_results]),
            'overall_best_mrr': max([r['mrr'] for r in all_results])
        }
    else:
        all_results = []
        for epoch in epoch_results:
            a1s = [r['a1'] for r in epoch_results[epoch]]
            a3s = [r['a3'] for r in epoch_results[epoch]]
            a5s = [r['a5'] for r in epoch_results[epoch]]
            a10s = [r['a10'] for r in epoch_results[epoch]]
            f1s = [r['f1'] for r in epoch_results[epoch]]
            mrrs = [r['mrr'] for r in epoch_results[epoch]]
            epoch_averages[epoch] = {
                'avg_a1': np.mean(a1s),
                'avg_a3': np.mean(a3s),
                'avg_a5': np.mean(a5s),
                'avg_a10': np.mean(a10s),
                'avg_f1': np.mean(f1s),
                'avg_mrr': np.mean(mrrs)
            }
            all_results.extend(epoch_results[epoch])
            if epoch_averages[epoch]['avg_f1'] > best_f1:
                best_f1 = epoch_averages[epoch]['avg_f1']
                best_epoch = epoch

        # Store best epoch results and overall max metrics
        best_results = {
            'best_epoch': best_epoch,
            'best_epoch_avg_a1': epoch_averages[best_epoch]['avg_a1'],
            'best_epoch_avg_a3': epoch_averages[best_epoch]['avg_a3'],
            'best_epoch_avg_a5': epoch_averages[best_epoch]['avg_a5'],
            'best_epoch_avg_a10': epoch_averages[best_epoch]['avg_a10'],
            'best_epoch_avg_f1': epoch_averages[best_epoch]['avg_f1'],
            'best_epoch_avg_mrr': epoch_averages[best_epoch]['avg_mrr'],
            'overall_best_a1': max([r['a1'] for r in all_results]),
            'overall_best_a3': max([r['a3'] for r in all_results]),
            'overall_best_a5': max([r['a5'] for r in all_results]),
            'overall_best_a10': max([r['a10'] for r in all_results]),
            'overall_best_f1': max([r['f1'] for r in all_results]),
            'overall_best_mrr': max([r['mrr'] for r in all_results])
        }

    # Save per-epoch averages and best results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    np.save(f'{result_dir}/epoch_averages_{timestamp}.npy', epoch_averages)
    np.save(f'{result_dir}/best_results_{timestamp}.npy', best_results)
    return best_results

def run_training(pretrain_sv_path, result_subdir):
    result_dir = f'/content/drive/MyDrive/USPM_edege_add/result/{result_subdir}'
    os.makedirs(result_dir, exist_ok=True)

    for downstream in ['function', 'poi']:
        print(f"\nStarting {downstream} downstream with {pretrain_sv_path}")
        args.downstream = downstream
        args.pretrain_sv_path = pretrain_sv_path
        # Set pretrain_scn_path based on downstream task
        if downstream == 'function':
            args.pretrain_scn_path = '/content/drive/MyDrive/USPM_edege_add/embeddings/qwen_text_embedding_function_72.pt'
        else:  # downstream == 'poi'
            args.pretrain_scn_path = '/content/drive/MyDrive/USPM_edege_add/embeddings/qwen_text_embedding_poi_72.pt'
        args.current_epoch = 0

        # 打印触发条件信息和正负样本配置
        if downstream == 'poi':
            print(f"InfoNCE Configuration:")
            print(f"  - Trigger: num >= 4 and f1 >= 0.38")
            print(f"  - Negative samples: similarity ∈ [0.1, 0.5], max_negatives=768")
            print(f"  - Positive samples: similarity > 0.9, max_positives=50")
            print(f"  - Extra positives per node: 3 (excluding existing neighbors)")
        else:
            print(f"InfoNCE Configuration:")
            print(f"  - Trigger: num >= 10 and f1 >= 0.44")
            print(f"  - Negative samples: similarity ∈ [0.1, 0.5], max_negatives=768")
            print(f"  - Positive samples: similarity > 0.9, max_positives=50")
            print(f"  - Extra positives per node: 3 (excluding existing neighbors)")

        for round_num in range(args.rounds):
        # for round_num in range(4, 5):
            print(f"\nRound {round_num + 1}/{args.rounds}")

            # 设置随机种子，确保可复现性
            base_seed = args.seed + round_num
            np.random.seed(base_seed)
            random.seed(base_seed + 1)
            torch.manual_seed(base_seed + 2)
            torch.cuda.manual_seed(base_seed + 3)
            torch.backends.cudnn.deterministic = True
            print(f"Random seeds set: numpy={base_seed}, random={base_seed+1}, torch={base_seed+2}, cuda={base_seed+3}")

            model = SV_GAT(args)
            model = model.to(args.device)

            opt1 = torch.optim.Adam(
                itertools.chain(model.attention_soft.parameters()),
                lr=0.0005, weight_decay=1e-8)
            opt4 = torch.optim.Adam(
                model.text_mlp.parameters(),
                lr=0.0005, weight_decay=1e-8)  # MLP和attention一样的设置
            if args.downstream == 'poi':
                opt3 = torch.optim.Adam(model.gat_poi.parameters(), lr=0.0005, weight_decay=5e-4)
                args.epochs = 250
            else:
                opt3 = torch.optim.Adam(model.gat.parameters(), lr=0.005, weight_decay=5e-4)
                args.epochs = 250

            if args.mode != 'mean':
                opt2 = torch.optim.SGD(model.sv_agg.parameters(), lr=0.005, weight_decay=1e-4, momentum=0.9)
                t = 10
                T = 800
                n_t = 0.5
                lf = lambda epoch: (0.9 * epoch / t + 0.1) if epoch < t else 0.1 if n_t * (
                        1 + math.cos(math.pi * (epoch - t) / (T - t))) < 0.1 else n_t * (
                        1 + math.cos(math.pi * (epoch - t) / (T - t)))
                scheduler = torch.optim.lr_scheduler.LambdaLR(opt2, lr_lambda=lf)
            else:
                opt2 = torch.optim.SGD(model.sv_agg.parameters(), lr=0.005, weight_decay=1e-4, momentum=0.9)

            print(model)

            # Collect results for this round
            round_results = {'round': round_num, 'epochs': []}
            last_test_result = None  # 保存上一次测试结果

            for epoch in range(args.start_epoch, args.epochs):
                # 在训练时传入上一次的测试结果，以便检查InfoNCE触发条件
                loss_epoch, pred_, street_embedding = trainer(args, model, opt1, opt2, opt3, opt4, epoch, last_test_result)
                if args.mode != 'mean':
                    scheduler.step()
                if epoch % args.print_num == 0:
                    result_tuple = test(args, model, epoch, round_num, f'{result_dir}/{downstream}')
                    # Append result to round_results
                    round_results['epochs'].append(result_tuple[-1])  # Last element is the result dict
                    last_test_result = result_tuple[-1]  # 保存当前测试结果

            # Save all results for this round in a single file
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            np.save(f'{result_dir}/{downstream}/round_{round_num}_{timestamp}.npy', round_results)

        # Calculate and save average and best results
        best_results = calculate_best_results(f'{result_dir}/{downstream}', downstream)
        print(f"Best Results for {downstream}:", best_results)

# 运行训练
if __name__ == "__main__":
    print("Enhanced InfoNCE with Positive and Negative Sampling")
    print("="*50)
    run_training('/content/drive/MyDrive/USPM_edege_add/embeddings/image_representation_117144_16.pt',
                 'infonce_pos3_neg_256_0.2')

Enhanced InfoNCE with Positive and Negative Sampling

Starting function downstream with /content/drive/MyDrive/USPM_edege_add/embeddings/image_representation_117144_16.pt
InfoNCE Configuration:
  - Trigger: num >= 10 and f1 >= 0.44
  - Negative samples: similarity ∈ [0.1, 0.5], max_negatives=768
  - Positive samples: similarity > 0.9, max_positives=50
  - Extra positives per node: 3 (excluding existing neighbors)

Round 1/5
Random seeds set: numpy=42, random=43, torch=44, cuda=45
SV_GAT(
  (text_mlp): Text_MLP(
    (layer1): Linear(in_features=4096, out_features=2048, bias=True)
    (relu): ReLU()
    (layer2): Linear(in_features=2048, out_features=768, bias=True)
  )
  (sv_agg): SVFeatureBlock(
    (lstm): LSTM(768, 768, batch_first=True)
  )
  (attention_soft): Attention_Soft(
    (l1): Linear(in_features=768, out_features=32, bias=True)
    (ac): Sigmoid()
    (l2): Linear(in_features=768, out_features=32, bias=False)
    (l3): Linear(in_features=32, out_features=1, bias=False)
  )


In [7]:
import os

# 检查传入的文件路径
file_path = '/content/drive/MyDrive/USPM_edege_add/embeddings/image_representation_117144_16.pt'
print("目标文件是否存在：", os.path.exists(file_path))

# 检查错误信息中的文件路径
error_file = 'embeddings/qwen_text_embedding_function_72.p'
print("错误文件是否存在：", os.path.exists(error_file))

目标文件是否存在： True
错误文件是否存在： False
